In [ ]:
!apt-get update
!apt install -y firefox
!wget https://github.com/mozilla/geckodriver/releases/download/v0.34.0/geckodriver-v0.34.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.34.0-linux64.tar.gz
!mv geckodriver /usr/local/bin/
!pip install selenium
!pip install geckodriver

from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import re
import numpy as np
import pandas as pd

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
--2024-02-19 11:21:32--  https://github.com/mozilla/geckodriver/releases/download/v0.34.0/geckodriver-v0.34.0-linux64.tar.gz
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/25354393/c74e12d7-7166-4aaa-9d7a-bbb01471db75?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz

/tmp/ipykernel_14825/2955951613.py:19: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
#Extracting sanrafael data

class SanRafaelDataExtractor:
    def __init__(self):
        self.options = webdriver.FirefoxOptions()
        self.options.add_argument('--headless')
        self.driver = webdriver.Firefox(options=self.options)

    def extract_data(self):
        # Get the webpage
        self.driver.get("https://www.cityofsanrafael.org/major-planning-projects-2/")

        self.driver.execute_script("window.scrollBy(0, 500);")
        sanrafael_table = self.driver.find_element('css selector', 'tbody')
        sanrafael_rows = sanrafael_table.find_elements('css selector', "tr")
        data = []
        for row in sanrafael_rows[1:]:
            cells = row.find_elements('css selector', "td")
            row_data = [cell.text for cell in cells]
            data.append(row_data)
        return data

    def save_to_csv(self, data, headers):
        with open('sanrafael.csv', 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(headers)
            csv_writer.writerows(data)

    def cleanup_data(self, data):
        #Create an empty dataframe with the desired columns
        sanrafael_df = pd.DataFrame(columns=['original_id','aug_id','country_name','country_code','map_coordinates','url','region_name','region_code','title', 'description', 'status','stages','date','procurementMethod','budget','currency','buyer','sector','subsector'])
        sanrafael_csv = pd.DataFrame(data, columns=['title', 'description', 'floor area', 'Number Units', 'BMR Units', 'applicant', 'staff', 'status'])
        sanrafael_csv.drop(['floor area', 'Number Units', 'BMR Units', 'applicant', 'staff'], axis=1, inplace=True)
        columns = ['title','description','status']
        for column in columns:
            sanrafael_df[column]=sanrafael_csv[column]
        return sanrafael_df

    def close_driver(self):
        self.driver.quit()

if __name__ == "__main__":
    headers = ['title','description','floor area','Number Units','BMR Units','applicant','staff','status']
    extractor = SanRafaelDataExtractor()
    data = extractor.extract_data()
    extractor.save_to_csv(data, headers)
    modified_data = extractor.cleanup_data(data)
    modified_data.to_csv('modified_sanrafael.csv', index=False, na_rep='NaN')
    extractor.close_driver()


In [ ]:
# Extracting Fairfield data

class FairFieldDataExtractor:
  def __init__(self):
    self.options=webdriver.FirefoxOptions()
    self.options.add_argument("--headless")
    self.driver = webdriver.Firefox(options=self.options)

  def extract_data(self):
    # Get the webpage
    self.driver.get("https://www.fairfield.ca.gov/government/city-departments/community-development/planning-division/development-activity?locale=en,https://www.fairfield.ca.gov/government/city-departments/public-works/capital-improvement-projects")
    # Close the pop-up
    self.driver.find_element('css selector', '.prefix-overlay-close.prefix-overlay-action-later').click()
    # Scroll down by 500 pixels
    self.driver.execute_script("window.scrollBy(0,500)")
    # Find the table using CSS Selector
    fairfield_table = self.driver.find_element('css selector', "tbody")
    # Get all the rows from the table using find element
    fairfield_rows = fairfield_table.find_elements('css selector', "tr")
    fair_data=[]
    # Iterate through the rows and extract data
    for row in fairfield_rows:
      # Get all cells present in the current row
      cells = row.find_elements('css selector', "td")
      # Extract and write data from each cell to csv file
      row_data = [cell.text for cell in cells]
      fair_data.append(row_data)
    return fair_data

  def save_to_csv(self,data,headers):
    # Create a csv file to store results
    with open('fairfield.csv', 'w', newline='') as csvfile:
      csv_writer = csv.writer(csvfile)
      csv_writer.writerow(headers)
      csv_writer.writerows(data)

  def cleanup_data(self,data):
    #Create an empty dataframe with the desired columns
    fairfield_df = pd.DataFrame(columns=['original_id','aug_id','country_name','country_code','map_coordinates','url','region_name','region_code','title', 'description', 'status','stages','date','procurementMethod','budget','currency','buyer','sector','subsector'])
    #read the csv file to dataframe
    fairfield_csv = pd.read_csv('fairfield.csv')
    #drop the unnecessary rows
    fairfield_csv.drop([0,1],axis=0,inplace=True)
    #mapping columns
    columns = ['original_id','title','location','subsector']
    for column in columns:
      fairfield_df[column]=fairfield_csv[column]
    return fairfield_df

  def close_driver(self):
    self.driver.quit()


if __name__ == "__main__":
    headers = ['original_id','title','location','subsector']
    fairfield_extractor = FairFieldDataExtractor()
    fairfield_data = fairfield_extractor.extract_data()
    fairfield_extractor.save_to_csv(fairfield_data, headers)
    fairfield_modified_data = fairfield_extractor.cleanup_data(fairfield_data)
    fairfield_modified_data.to_csv('modified_fairfield.csv', index=False, na_rep='NaN')
    fairfield_extractor.close_driver()


In [ ]:
#extracting elk
class ElkGroveDataExtractor:
  def __init__(self):
    self.options= webdriver.FirefoxOptions()
    self.options.add_argument("--headless")
    self.driver= webdriver.Firefox(options =self.options)

  def extract_data(self):
    # Get the webpage
    self.driver.get("https://www.elkgrovecity.org/southeast-policy-area/development-projects")
    #Get all rows
    elk_rows=self.driver.find_elements('css selector',"tr")
    elk_data = []
    for row in elk_rows:
      # Get all the cells present in the row
      cells = row.find_elements('css selector', "td")
      # Extract each cell and write the data to csv
      row_data = [cell.text for cell in cells]
      elk_data.append(row_data)
    return elk_data

  def save_to_csv(self,data,headers):
    with open('elk.csv', 'w', newline='') as csvfile:
       csv_writer = csv.writer(csvfile)
       self.driver.execute_script("window.scrollBy(0,550)")
       csv_writer.writerow(headers)
       csv_writer.writerows(data)

  def cleanup_data(self,data):
    # Create an empty dataframe with the desired columns
    elk_df = pd.DataFrame(columns=['original_id','aug_id','country_name','country_code','map_coordinates','url','region_name','region_code','title', 'description', 'status','stages','date','procurementMethod','budget','currency','buyer','sector','subsector'])
    #read the csv file to dataframe
    elk_csv = pd.read_csv('elk.csv')
    #mapping columns
    mapping_columns = ['title','description','status']
    for column in mapping_columns:
      elk_df[column] = elk_csv[column]
    return elk_df

  def close_driver(self):
    self.driver.quit()

if __name__ == "__main__":
  headers = ['project_code','title','description','applicant','status','project_materials']
  elkgrover_extractor = ElkGroveDataExtractor()
  elkgrover_data = elkgrover_extractor.extract_data()
  elkgrover_extractor.save_to_csv(elkgrover_data,headers)
  modified_data = elkgrover_extractor.cleanup_data(elkgrover_data)
  modified_data.to_csv('modified_elk.csv',index=False,na_rep='NaN')
  elkgrover_extractor.close_driver()

In [ ]:
#eracting arcata data

class ArcataDataExtractor:
  def __init__(self):
      self.options = webdriver.FirefoxOptions()
      self.options.add_argument("--headless")
      self.driver = webdriver.Firefox(options=self.options)

  def extract_data(self):
      # Create an empty dataframe with the desired columns
      arcata_df = pd.DataFrame(columns=['original_id', 'aug_id', 'country_name', 'country_code', 'map_coordinates', 'url', 'region_name', 'region_code', 'title', 'description', 'status', 'stages', 'date', 'procurementMethod', 'budget', 'currency', 'buyer', 'sector', 'subsector'])
      # get the webpage
      self.driver.get("https://www.cityofarcata.org/413/Current-City-Construction-Projects")
      # find the table with CSS selector
      arcata_table = self.driver.find_element('css selector', "div[class='widgetBody'] table")
      # Get all the rows from the table
      arcata_rows = arcata_table.find_elements('css selector', "tr")
      # Iterate through the row and accumulate title, description, status
      for i in range(1, len(arcata_rows)):
          # Scroll down the page by 100 pixels
          self.driver.execute_script("window.scrollBy(0,100)")
          # extract the project title
          project_title = self.driver.find_element('css selector', f'tbody tr:nth-child({i}) td:nth-child(1) strong:nth-child(1)')
          # extract the description
          project_description = self.driver.find_element('css selector', f'tbody tr:nth-child({i}) td:nth-child(2)')
          # extract the status
          project_status = self.driver.find_element('css selector', f'tbody tr:nth-child({i}) td:nth-child(3)')
          # regex pattern to extract budget
          budget_pattern = r'\$(\d+(?:,\d{3})*(?:\.\d+)?)(?:\s*(million|billion|thousand))?'
          # Search for the pattern in the text
          bud_match = re.search(budget_pattern, project_description.text)
          if bud_match:
              project_budget = bud_match.group(0)
          else:
              project_budget = "null"
          # regex pattern to extract buyer
          buyer_pattern = r'contracted with\s+([^\d.,;]+)\b'
          # Find matches in the text
          buy_match = re.search(buyer_pattern, project_description.text)
          # Print the matches
          if buy_match:
              project_buyer = buy_match.group(0)
          else:
              project_buyer = "null"
          arcata_df = arcata_df._append({"title": project_title.text, "description": project_description.text,
                                          "status": project_status.text, "budget": project_budget,
                                          "buyer": project_buyer}, ignore_index=True)
      # Replace empty values with NaN
      arcata_df = arcata_df.replace(r'^\s*$', np.nan, regex=True)
      return arcata_df

  def close_driver(self):
      self.driver.quit()

if __name__ == "__main__":
  arcata_extractor = ArcataDataExtractor()
  arcata_data = arcata_extractor.extract_data()
  arcata_data.to_csv('arcata.csv', index=False, na_rep='NaN')
  arcata_extractor.close_driver()


/tmp/ipykernel_19353/98782265.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  arcata_df = arcata_df.replace(r'^\s*$', np.nan, regex=True)


In [ ]:
#extracting paso data

class PasoDataExtractor:
    def __init__(self):
        self.options = webdriver.FirefoxOptions()
        self.options.add_argument("--headless")
        self.driver = webdriver.Firefox()

    def extract_data(self):
        # Get the webpage
        self.driver.get("https://www.prcity.com/363/City-Projects")
        # Create an empty DataFrame
        paso_df = pd.DataFrame(columns=['original_id', 'aug_id', 'country_name', 'country_code', 'map_coordinates', 'url', 'region_name', 'region_code', 'title', 'description', 'status', 'stages', 'date', 'procurementMethod', 'budget', 'currency', 'buyer', 'sector', 'subsector'])
        # Scroll down the page by 500 pixels
        self.driver.execute_script("window.scrollBy(0,500)")
        # Click on the 'Out for Bid'
        self.driver.find_element('css selector', ".Hyperlink[href='/392/Out-to-Bid']").click()
        self.driver.implicitly_wait(10)
        # Scroll down the page by 500 pixels
        self.driver.execute_script("window.scrollBy(0,500)")
        # Click on Show Me option
        self.driver.find_element('css selector', "#CatID").click()
        # Select 'All Bids' option
        self.driver.find_element('css selector', "option[value='All']").click()
        # Wait until sort by dropdown is visible
        sort_dropdown = WebDriverWait(self.driver, 20).until(
        EC.visibility_of_element_located((By.ID, "txtSort"))
        )
        # Create action chain object
        action = ActionChains(self.driver)
        # Click on the sort_dropdown menu
        action.move_to_element(sort_dropdown).click().perform()
        # Select 'Closing Date' option
        self.driver.find_element('css selector', "option[value='Date']").click()
        # Filter the bids which are in open status
        self.driver.find_element('css selector', "#showAllBids").click()
        # Scroll down the page by 50 pixels
        self.driver.execute_script("window.scrollBy(0,400)")
        # Find the table using CSS Selector
        paso_table = self.driver.find_element('css selector', ".bidItems")
        # Fetch the rows in the table
        paso_rows = paso_table.find_elements('css selector', "tr")
        project_titles = []
        #Iterating through all the rows and storing the project title
        for row in paso_rows:
            links = row.find_elements('css selector', "a")
            for link in links:
                if link.text != "Read on":
                    project_titles.append(link.text)
        for link in project_titles:
            self.driver.execute_script("window.scrollBy(0,150)")
            xpath = f'//a[contains(text(), "{link}")]'
            #click on the project title
            self.driver.find_element(By.XPATH, xpath).click()
            #extract the description details
            details=self.driver.find_element('css selector',"tbody tr p:nth-child(1)")
            #extract the  publication details
            publication_details =self.driver.find_element('css selector',"body > div:nth-child(26) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > form:nth-child(2) > div:nth-child(12) > table:nth-child(5) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(2) > div:nth-child(3) > div:nth-child(3) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(4) > td:nth-child(1) > span:nth-child(1)")
            #parse the publications details to date time object
            date_object  = datetime.strptime(publication_details .text, "%m/%d/%Y %I:%M %p")
            #xtract date
            publication_date = date_object.date()
            #extract status
            status = self.driver.find_element(By.XPATH,'//span[normalize-space()="Open"]')
            paso_df = paso_df._append({'title': link, 'description': details.text,'status':status.text, 'date': publication_date}, ignore_index=True)
            self.driver.back()

        # Replace empty values with NaN
        paso_df = paso_df.replace(r'^\s*$', np.nan, regex=True)
        return paso_df

    def close_driver(self):
      self.driver.quit()

if __name__=="__main__":
  paso_extractor = PasoDataExtractor()
  paso_data=paso_extractor.extract_data()
  paso_data.to_csv('modified_paso.csv',index=False,na_rep='NaN')
  paso_extractor.close_driver()


/tmp/ipykernel_14825/2851960632.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  paso_df = paso_df.replace(r'^\s*$', np.nan, regex=True)
